<a href="https://colab.research.google.com/github/brunoodon/4_workshop_misp/blob/main/Hunting_automatizado_de_CVE_recentemente_exploradas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#instalando o módulo PyMISP
!pip3 install pymisp

In [ ]:
#importando as bibliotecas necessárias
import pandas as pd
import requests
import numpy as np
import csv
import json
import datetime
import time
import statistics
from pandas.io.parsers import python_parser
from pymisp import MISPEvent, MISPObject, PyMISP, ExpandedPyMISP, MISPSharingGroup
#url dos exploits do CISA
url_cisa = 'https://www.cisa.gov/sites/default/files/csv/known_exploited_vulnerabilities.csv'
#url do inventário de ativos do cliente
url_inventory = 'https://raw.githubusercontent.com/brunoodon/4_workshop_misp/main/assets.csv'
#preencha com a URL da sua instância do MISP
url_misp = 'MISP_URL'
#Preencha com a authkey de um usuário do MISP com privilégios para criar eventos e inserir atributos.
misp_key = 'MISP_APIKEY'
#Preencha com a url do Elastic
url_es = 'ELASTIC_URL:9200/alerts/_doc'

In [ ]:
#lendo e dando 'print' no dataset do inventário do ativos do cliente
inventory = pd.read_csv(url_inventory)
display(inventory)

In [ ]:
#lendo e dando 'print' no dataset de vulnerabilidades com exploit recente
exploited_cve = pd.read_csv(url_cisa)
display(exploited_cve)

In [ ]:
#percorrendo cada linha do dataset de inventário de ativos do cliente
for a in range(len(inventory)):
  application=inventory.values[a][0]
  version = inventory.values[a][1]
  ip=inventory.values[a][2]
  hostname=inventory.values[a][3]
  dep=inventory.values[a][4]
  #criando uma variável com a data atual para inserirmos como parte do nome do evento
  today=str(datetime.date.today())
  #fazendo a busca dos registros do dataset de vulnerabilidades exploradas onde o campo 'Product' contém o nome da aplicação descrita em cada linha de inventário
  search = exploited_cve.query('product.str.contains("'+application+'") == True', engine="python")
  misp_verifycert = False
  #definindo as características do evento do MISP
  misp = ExpandedPyMISP(url_misp, misp_key, misp_verifycert)
  event = MISPEvent()
  event.info = f"Risco de vulnerabilidades no ativo {hostname} | {application} - {version} - "+today+""
  event.analysis = "2"
  event.published = True
  event.distribution = "4" #compartilhar com Sharing group
  event.sharing_group_id = "1" #ID do Sharing group que receberá o evento
  event.threat_level_id = "1" #level HIGH
  event.add_tag('Exploited_CVE')#uma tag criada para identificar que este evento tem CVE com exploit
  event.add_tag('tlp:amber+strict') #só a entidade que enviou e a entidade que recebeu podem ter acesso ao avento
  event.add_tag('circl:incident-classification="vulnerability"')#tag que identifica o tipo de ameaça que será compartilhada. Neste caso, uma tag da taxonomia do CIRCL.
    #percorrendo cada resultado dessa busca e atribuíndo os valores às variáveis
  for b in range(len(search)):
      cve=search.values[b][0]
      vendor=search.values[b][1]
      product=search.values[b][2]
      vulnerability_name=search.values[b][3]
      date_added=search.values[b][4]
      short_desc=search.values[b][5]
      due_date=search.values[b][6]
      notes=search.values[b][7]
      #fazendo a consulta do CVE no banco do NVD, para enriquecer o dataset
      url_nvd = 'https://services.nvd.nist.gov/rest/json/cves/2.0?cveId='+cve+''
      r_nvd = requests.get(url_nvd)
      #foi necessário um temporizador porque a API do NVD tem limitação de requests por minuto
      time.sleep(8)
      try:
        #lendo o json com o resultado e atribuindo cada resultado de campo às varáveis severity_score e vector_string
        #obs: foi preciso estabelecer 2 possíveis estruturas, uma para cada versão da métrica de severity score (V2 ou V3.1)
        json_nvd = json.loads(r_nvd.text)
        for c in json_nvd['vulnerabilities']:
          cpe = []
          try:
            for d in c['cve']['metrics']['cvssMetricV31']:
              vector_string = d['cvssData']['vectorString']
              #convertendo o campo de severity_score em float
              severity_score = float(d['cvssData']['baseScore'])
          except:
            for d in c['cve']['metrics']['cvssMetricV2']:
              vector_string = d['cvssData']['vectorString']
              severity_score = float(d['cvssData']['baseScore'])
          for b in c['cve']['configurations']:
            for x in b['nodes']:
              for z in x['cpeMatch']:
                #verificando se a versão da aplicação foi encontrada em algum CPE referente à vulnerabilidade
                if version in z['criteria']:
                  #se for o caso, inserir o CPE na lista
                  cpe.append(z['criteria'])
          #só adiciona a vulnerabilidade no Elastic e no MISP caso o CPE não seja vazio
          if cpe != []:
            #adicionando as CVE, com os detalhes no comentário
            event.add_attribute('vulnerability', str(cve), disable_correlation=True, to_ids=False, comment=f'Severity Score: {severity_score}|Vector String:{vector_string}|Product: {product}')
            print(f'Adicionando a CVE {cve} no evento')
            #Definindo o header e o documento JSON que será incluído no Elasticsearch
            es_header = {'Content-Type': 'application/json'}
            es_json = {
            'Vulnerability': cve,
            'VulnerabilityName': vulnerability_name,
            'Description': short_desc,
            'IP': ip,
            'CPE': cpe,
            'Vendor': vendor,
            'Hostname': hostname,
            'Application': f'{application}:{version}',
            'ExploitationVector': vector_string,
            'SeverityScore': float(severity_score),
            'AlertDate': today,
            'Department': dep
            }
            #Inserindo o alerta de possível vulnerabilidade em um índex do Elasticsearch
            r_es = requests.post(url_es, headers=es_header, verify=False, json=es_json)
      except:
        print('Nenhuma vulnerabilidade encontrada no ativo')
  event = misp.add_event(event)